In [4]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_chroma import Chroma

DATA_PATH = "data"

# Document LOADER
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

# Split Documents into Chunks
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM

# Multi Query: Different Perspectives
perspectives_template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

# RAG-Fusion: Related
fusion_template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""


prompt_perspectives = ChatPromptTemplate.from_template(perspectives_template)
prompt_rag_fusion = ChatPromptTemplate.from_template(fusion_template)


from langchain_core.output_parsers import StrOutputParser



generate_queries = (
    prompt_rag_fusion 
    | OllamaLLM(model="llama3.2")
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [6]:
from langchain_ollama import OllamaEmbeddings

#### INDEXING ####
documents = load_documents()
splits = split_documents(documents)

from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text"))

# Define Retriever 
retriever = vectorstore.as_retriever()

In [7]:
from langchain.load import dumps, loads

# Define function to remove duplicates from the multiple queries outputs
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

# Retrieve
question = "What is perimeter fit?"

# MULTI_QUERY CHAIN
#retrieval_chain = generate_queries | retriever.map() | get_unique_union
# RAG FUSION CHAIN
retrieval_chain = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain.invoke({"question":question})
len(docs)


C:\Users\eriko\AppData\Local\Temp\ipykernel_3216\2298845656.py:38: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


8

In [8]:
from operator import itemgetter
from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="llama3.2")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Perimeter Fit is a novel module introduced in the ReFit framework for semantic segmentation. It uses object perimeters to improve saliency and refine Class Activation Map (CAM) predictions before using pixel-similarity-based refinement. The main goal of Perimeter Fit is to increase the quality of CAM predictions while simultaneously refining the training labels for state-of-the-art Fully Segmented Surface Segmentation (FSSS) models.'